# Train

In [ ]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
import glob
import numpy as np
import os
import io
import requests
from torch.autograd import Variable


# *********************************************************Load the data***************************************************

# data_paths 
data_dir = 'flowers'

train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'



# Data transforms
data_transforms = {
    'train': transforms.Compose([transforms.RandomRotation(25),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])]),
    'valid' : transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])]),
    'test' : transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
}
train_transforms = data_transforms['train']
valid_transforms = data_transforms['valid']
test_transforms =data_transforms['test']



# Image Datasets
image_datasets = {
    'train' : datasets.ImageFolder(train_dir,transform=train_transforms),
    'valid': datasets.ImageFolder(valid_dir,transform=valid_transforms),
    'test' : datasets.ImageFolder(test_dir,transform=test_transforms)
}
train_dataset =image_datasets['train']
valid_dataset =image_datasets['valid']
test_dataset = image_datasets['test']



# Data loaders 
data_loaders = {
    'train' : torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True),
    'valid': torch.utils.data.DataLoader(valid_dataset,batch_size=64),
    'test' : torch.utils.data.DataLoader(test_dataset,batch_size=64)    
}
train_loader = data_loaders['train']
valid_loader = data_loaders['valid']
test_loader  = data_loaders['test']


# *********************************************************Label Mapping***************************************************

import json
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
print("training done")


# ******************************************Building and training the classifier*******************************************


# ******************************************Building the classifier*******************************************
# selecting "densenet121" as the pretrained model as the number of inputs is less 1024 but contain more hidden layers

n_inputs = 1024 ## will get from the pretrained model
n_outputs = 102 # as defined in the dataset total of 102 different flower types

# Defining the model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters(): # Freeze parameters so we don't backprop through them
    param.requires_grad = False

    
# Defining our classifier for the pretrained model
Classifier = nn.Sequential(nn.Linear(n_inputs,800),
                                 nn.ReLU(),
                                 nn.Dropout(0.3),
                                 nn.Linear(800,512),
                                 nn.ReLU(),
                                 nn.Dropout(0.4),
                                 nn.Linear(512, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.3),
                                 nn.Linear(256, n_outputs),
                                  nn.LogSoftmax(dim=1))

model.classifier = Classifier

criterion = nn.NLLLoss() # Defining the loss as NLLLoss

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001) 


# Move input and label tensors to the GPU and also the model
for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    
model = model.to(device)

print("training done")

# ******************************************Training the classifier*******************************************


def train_the_model(model,criterion,optimizer,train_loader,valid_loader,epochs):
    steps = 0
    running_loss = 0
    print_every = 5
    train_losses, valid_losses = [], []
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            steps += 1
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                valid_loss = 0
                accuracy = 0
                model.eval()
                with torch.no_grad():
                    for inputs, labels in valid_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        logps = model.forward(inputs)
                        batch_loss = criterion(logps, labels)

                        valid_loss += batch_loss.item()

                        # Calculate accuracy
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                train_losses.append(running_loss/print_every)
                valid_losses.append(valid_loss/len(valid_loader))                    
                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Valid loss: {valid_loss/len(valid_loader):.3f}.. "
                      f"Valid accuracy: {accuracy/len(valid_loader):.3f}")
                running_loss = 0
                model.train()

    return model
epochs = 7
model = train_the_model(model,criterion,optimizer,train_loader,valid_loader,epochs)

  
# To view the performance in a graph
# plt.plot(train_losses, label='Training loss')
# plt.plot(valid_losses, label='Validation loss')
# plt.legend(frameon=False)


# ******************************************Testing the classifier*******************************************

def testing_model(model,test_loader,criterion):
    test_loss = 0
    accuracy = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    print(f"Test loss: {test_loss/len(test_loader):.3f}.. "
                f"Test accuracy: {100*(accuracy/len(test_loader)):.3f}")
    return test_loss/len(test_loader),100*(accuracy/len(test_loader))


test_loss,test_accuracy = testing_model(model,test_loader,criterion)





# ******************************************Saving the Checkpoint*******************************************


model.class_to_idx = train_dataset.class_to_idx
class_names = train_dataset.classes
saved_model = {'input_size':n_inputs,
               'output_size':n_outputs,
               'epochs': 5,
               'batch_size':64,
               'model' : models.densenet121(pretrained=True),
               'Classifier': Classifier,
               'optimizer' : optimizer.state_dict(),
               'state_dict':model.state_dict(),
               'class_to_idx': model.class_to_idx,
               'class_names' : class_names
}
torch.save(saved_model,"saved_model.pth")


print("model trained Saved Successfully with  accuracy", test_accuracy)



training done


/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


training done
Epoch 1/7.. Train loss: 4.626.. Valid loss: 4.577.. Valid accuracy: 0.028
Epoch 1/7.. Train loss: 4.544.. Valid loss: 4.531.. Valid accuracy: 0.025
Epoch 1/7.. Train loss: 4.485.. Valid loss: 4.494.. Valid accuracy: 0.056
Epoch 1/7.. Train loss: 4.554.. Valid loss: 4.487.. Valid accuracy: 0.056
Epoch 1/7.. Train loss: 4.523.. Valid loss: 4.479.. Valid accuracy: 0.055
Epoch 1/7.. Train loss: 4.489.. Valid loss: 4.442.. Valid accuracy: 0.026
Epoch 1/7.. Train loss: 4.483.. Valid loss: 4.416.. Valid accuracy: 0.025
Epoch 1/7.. Train loss: 4.434.. Valid loss: 4.374.. Valid accuracy: 0.055
Epoch 1/7.. Train loss: 4.384.. Valid loss: 4.318.. Valid accuracy: 0.050
Epoch 1/7.. Train loss: 4.286.. Valid loss: 4.236.. Valid accuracy: 0.049
Epoch 1/7.. Train loss: 4.260.. Valid loss: 4.137.. Valid accuracy: 0.095
Epoch 1/7.. Train loss: 4.185.. Valid loss: 4.028.. Valid accuracy: 0.105
Epoch 1/7.. Train loss: 4.153.. Valid loss: 3.935.. Valid accuracy: 0.115
Epoch 1/7.. Train loss: 





# Predict

In [ ]:
# ******************************************Loading the Checkpoint*******************************************

device = torch.device("cuda")
loaded_model = torch.load('saved_model.pth')
def load_model(file_path,map_location='cpu'):
    
    loaded_model = torch.load(file_path)
    model1 = loaded_model['model']
    model1.classifier = loaded_model['Classifier']
    model1.load_state_dict(loaded_model['state_dict'],strict=False)
    model1.class_to_idx = loaded_model['class_to_idx']
    optimizer = loaded_model['optimizer']
    epochs = loaded_model['epochs']
    class_names = loaded_model['class_names']

    for param in model1.parameters():
        param.requires_grad = False
        
    return model1,loaded_model['class_to_idx'],class_names

model,class_to_idx,class_names = load_model('saved_model.pth')

model.to(device)


#******************************************Processing the input image*******************************************

def process_image(image):
    
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    transform = transforms.Compose([
                 transforms.Resize(256),
                 transforms.CenterCrop(224),
                 transforms.ToTensor(),
                 transforms.Normalize([0.485, 0.456, 0.406],
                                      [0.229, 0.224, 0.225])
                 ])

    return transform(image)



def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

# ******************************************Predicting the image*******************************************
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.eval()
    image = Image.open(image_path)
    img =process_image(image)
    
    img = np.expand_dims(img,0) # 2D to 1D
    
    img = torch.from_numpy(img)
    
    inputs = Variable(img).to(device)
    log_ps = model.forward(inputs)
    
    ps = F.softmax(log_ps,dim=1)
    topk = ps.cpu().topk(topk)
    
    return (i.data.numpy().squeeze().tolist() for i in topk)
    
    
image_path = "assets/Capture_test_flower.JPG"
probs, classes = predict(image_path, model)
print(classes)
print(probs)

#***************************Getting top 5 predicted names for the flower*****************************
flower_names= [cat_to_name[class_names[i]] for i in classes]
print(flower_names)


#******************************************Sanity Checking*******************************************

def view_classify(image_path,probs,classes,mapping):
    # to view image and get predicted class
    image = Image.open(image_path)
    
    fig,(ax1,ax2) = plt.subplots(figsize=(8,10),ncols=1,nrows=2)
    flower_name = flower_names[0]
    ax1.set_title(flower_name)
    ax1.imshow(image)
    ax1.axis('off')
    
    y_pos = np.arange(len(probs))
    ax2.barh(y_pos,probs,align='center')
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(flower_names)
    ax2.invert_yaxis()
    ax2.set_title('Class Probability')
    
view_classify(image_path,probs,classes,cat_to_name)
